In [27]:
from scipy.io import wavfile
import matplotlib.pyplot as plt
import os
import librosa
from PIL import Image
import re
from utils_com import *
import librosa.display

This code was designed by us, but have used ChatGPT for optimization, troubleshooting, and plots.

In [28]:
#Load data

window_size = 512

reconstructed_MNIST = f"baseline_recon/audio_mnist{window_size}"
MNIST_recon_files = os.listdir(reconstructed_MNIST)

original_files_mnist = []

for i in range(1,61):
    if i < 10: 
        dir_og = f"original audio/AMNISTdata/0{i}"
        dir_OG_files = os.listdir(dir_og)
        for file in dir_OG_files:
            if file in MNIST_recon_files:
                original_files_mnist.append(file)
    else:
        dir_og = f"original audio/AMNISTdata/{i}"
        dir_OG_files = os.listdir(dir_og)
        for file in dir_OG_files:
            if file in MNIST_recon_files:
                original_files_mnist.append(file)

In [29]:
#Original

# Clean the output directory before saving new plots
output_dir = f"baseline_results/time_plots/{window_size}/Original/"

if os.path.exists(output_dir):
    # Remove all files in the output directory
    for file in os.listdir(output_dir):
        file_path = os.path.join(output_dir, file)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f"Error deleting file {file_path}: {e}")
else:
    # Create the directory if it doesn't exist
    os.makedirs(output_dir)

# Get a list of all files in the folder_path
audio_dir = "original audio/AMNISTdata/"
all_files = original_files_mnist


# Process each file in the reconstruction directory
for file in all_files:

    folder_no = re.split("\_", file)[1]
    file_path = os.path.join(audio_dir, folder_no +"/"+ file)

    # Load the audio data
    sr , _ = wavfile.read(file_path)
    y, sr = librosa.load(file_path, sr = sr)

    # Create a plot
    plt.figure()
    plt.plot(y)
    plt.title(f"{file} (Original)")
    plt.xlabel('Time (samples)')
    plt.ylabel('Amplitude')
    
    # Set the y-axis limits to -0.35 to 0.35
    plt.ylim([-0.25, 0.25])
    
    
    # Save the plot to the specified folder
    plot_path = os.path.join(output_dir, f"{file}.png")
    plt.savefig(plot_path)
    plt.close()  # Close the figure to free up memory


In [30]:
#Reconstructed

# Clean the output directory before saving new plots
output_dir = f"baseline_results/time_plots/{window_size}/Reconstructed/"

if os.path.exists(output_dir):
    # Remove all files in the output directory
    for file in os.listdir(output_dir):
        file_path = os.path.join(output_dir, file)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f"Error deleting file {file_path}: {e}")
else:
    # Create the directory if it doesn't exist
    os.makedirs(output_dir)

# Get a list of all files in the folder_path
audio_dir = f"baseline_recon/audio_mnist{window_size}/"
# Filter for WAV files only
valid_extensions = ['.wav']
all_files = [f for f in MNIST_recon_files if f.endswith(tuple(valid_extensions))]


# Process each file in the reconstruction directory
for file in all_files:
    file_path = os.path.join(audio_dir, file)

    # Load the audio data
    try:
        y, sr = librosa.load(file_path, sr=None)  # Load using librosa
    except Exception as e:
            print(f"Error loading {file_path}: {e}")
            continue  # Skip to the next file

    y, sr = librosa.load(file_path, sr = sr)

    # Create a plot
    plt.figure()
    plt.plot(y)
    plt.title(f"{file} (Reconstructed, {window_size})")
    plt.xlabel('Time (samples)')
    plt.ylabel('Amplitude')
    
    # Set the y-axis limits to -0.35 to 0.35
    plt.ylim([-0.25, 0.25])
    
    # Save the plot to the specified folder
    plot_path = os.path.join(output_dir, f"{file}.png")
    plt.savefig(plot_path)
    plt.close()  # Close the figure to free up memory


In [31]:
# Time domain

audio_dir = f'baseline_results/time_plots/{window_size}/Original/'
recon_dir = f'baseline_results/time_plots/{window_size}/Reconstructed'

output_dir_time = f"baseline_results/time_plots/{window_size}/Comparison"

# Prepare the output directory
if os.path.exists(output_dir_time):
    # Remove all files in the output directory
    for f in os.listdir(output_dir_time):
        os.remove(os.path.join(output_dir_time, f))
else:
    # If the directory doesn't exist, create it
    os.makedirs(output_dir_time)

# List files in the reconstructed directory
files_com = os.listdir(recon_dir)

for file in files_com:
    # Construct the paths for the Original and Reconstructed files
    original_file_path = os.path.join(audio_dir, file)
    recon_file_path = os.path.join(recon_dir, file)

    # Check if the corresponding file exists in the Original directory
    if not os.path.isfile(original_file_path):
        continue  # Skip this file if it doesn't exist in Original

    # Open the images
    OG_path = Image.open(original_file_path)
    recon_path = Image.open(recon_file_path)

    # Get the size of each image
    (width1, height1) = OG_path.size
    (width2, height2) = recon_path.size

    # Create a new image with a width equal to the sum of both images' widths
    combined = Image.new('RGB', (width1 + width2, max(height1, height2)))

    # Paste the images into the new combined image
    combined.paste(OG_path, (0, 0))
    combined.paste(recon_path, (width1, 0))

    # Save the combined image to the output directory
    combined.save(os.path.join(output_dir_time, file))
